<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#LGBM---New-Feature-+-Baseline-Rolling-Window" data-toc-modified-id="LGBM---New-Feature-+-Baseline-Rolling-Window-1">LGBM - New Feature + Baseline Rolling Window</a></span><ul class="toc-item"><li><span><a href="#Defining-metrics" data-toc-modified-id="Defining-metrics-1.1">Defining metrics</a></span></li><li><span><a href="#Building-our-dataset" data-toc-modified-id="Building-our-dataset-1.2">Building our dataset</a></span></li><li><span><a href="#Feature-building" data-toc-modified-id="Feature-building-1.3">Feature building</a></span></li><li><span><a href="#Maximum-error" data-toc-modified-id="Maximum-error-1.4">Maximum error</a></span></li><li><span><a href="#Dataset-Splitting-(Train-until-week-3-/-Val.-week-2/-Test-week-1)" data-toc-modified-id="Dataset-Splitting-(Train-until-week-3-/-Val.-week-2/-Test-week-1)-1.5">Dataset Splitting (Train until week 3 / Val. week 2/ Test week 1)</a></span></li></ul></li><li><span><a href="#HOLY-FUCK" data-toc-modified-id="HOLY-FUCK-2">HOLY FUCK</a></span></li><li><span><a href="#FUCK-THE-ORIGINAL-MODEL,-LET'S-JUST-PREDICT-THE-NEW-ITEMS" data-toc-modified-id="FUCK-THE-ORIGINAL-MODEL,-LET'S-JUST-PREDICT-THE-NEW-ITEMS-3">FUCK THE ORIGINAL MODEL, LET'S JUST PREDICT THE NEW ITEMS</a></span></li><li><span><a href="#HAHAHAHAHAHAHAHAHAHAHAHA" data-toc-modified-id="HAHAHAHAHAHAHAHAHAHAHAHA-4">HAHAHAHAHAHAHAHAHAHAHAHA</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Utilities" data-toc-modified-id="Utilities-4.0.1">Utilities</a></span></li></ul></li></ul></li></ul></div>

# LGBM - New Feature + Baseline Rolling Window

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from utils import *
import sys
import xgboost as xgb
import lightgbm as lgb
from datetime import datetime

NUMBER_OF_LAGS = 4

sys.path.append("../main/datasets/")
!ls  ../main/datasets/

1.0v  1.0v.zip


<hr>

## Defining metrics

Baseline_score function

In [2]:
def baseline_score(prediction, target, simulatedPrice):
    prediction = prediction.astype(int)

    return np.sum((prediction - np.maximum(prediction - target, 0) * 1.6)  * simulatedPrice)

Evaluation Metric

In [3]:
def feval(prediction, dtrain):
    
    prediction = prediction.astype(int)
    target = dtrain.get_label()

    simulatedPrice = dtrain.get_weight()
    
    return 'feval', np.sum((prediction - np.maximum(prediction - target, 0) * 1.6)  * simulatedPrice), True

<hr>

## Building our dataset
This notebook makes this step cleaner than the previous versions. So It'll be tidier and shorter than before!

In [4]:
infos, items, orders = read_data("../main/datasets/")
print("Sanity checks...", infos.shape, items.shape, orders.shape)

Sanity checks... (10463, 3) (10463, 8) (2181955, 5)


In [5]:
# Changing our time signatures
process_time(orders)

In [6]:
df = dataset_builder(orders, items)

<hr>

## Feature building

**Adding 'is_new'**

In [7]:
# This cell adds a feature responsible for indicating if in the current week
# a given item has its first appearance.
orders_sorted_by_week = orders.sort_values('group_backwards', ascending=False)
weeks_grouped_by_items = orders_sorted_by_week.groupby('itemID', as_index=False)
items_first_appearance = weeks_grouped_by_items.first()[['itemID', 'group_backwards']]

items_first_appearance.rename(columns={'group_backwards':'first_appearance'}, inplace=True)

df['is_new'] = 0

df = pd.merge(df, items_first_appearance, left_on=['itemID'], right_on=['itemID'], how='left', validate='m:1')

df.loc[df['first_appearance'] == df['group_backwards'], 'is_new'] = 1
df.drop(columns=['first_appearance'], inplace=True)

**Cumulative sale by category**

In [8]:
# percentage_accum_cat_3 feature...
df = cumulative_sale_by_category(df)

**Time Encoding**

In [9]:
# Encoding our weeks as a series of sines and cosines...
# This function will consider our period as a semester in a year,
# so we can try other types of time encoding later!
df = time_encoder(df, 'group_backwards', 26)

**Lags and diffs**

In [10]:
# This cell lags and diffs our feature 'orderSum'
shifting = df.copy()

for i in range(1, NUMBER_OF_LAGS + 1):
    # Carrying the data of weeks t-1
    shifting[f'orderSum_{i}'] = shifting.groupby('itemID')['orderSum'].shift(i)

    
    # Getting the difference of the orders and promotions between weeks t-1 and t-2...
    shifting[f'orderSum_diff_{i}'] = shifting.groupby('itemID')[f'orderSum_{i}'].diff()
    

**Rolling window "orderSum"**

In [11]:
%%time
# This cell creates rolling-window features based on 'orderSum' in our dataset!
item_group = shifting.groupby(["itemID", "group_backwards"]).agg({'orderSum':'sum'})

# We'll .shift(-1) because it sorts our "group_backwards", 
# so doing .shift(1) would cause a HUGE dataleak.
aux_shifting = item_group.groupby('itemID')[['orderSum']].shift(-1)

aux_shifting.sort_values(['itemID', 'group_backwards'], ascending=[True, False], inplace=True)

for i in range(3):
    rolled_window = aux_shifting.groupby(['itemID'], as_index=False)[['orderSum']].rolling(2 ** i).mean()
    rolled_window.rename(columns={'orderSum':f"orderSum_mean_rolled_{i}"}, inplace=True)
    shifting = pd.merge(shifting, rolled_window, left_on=['itemID', 'group_backwards'], right_on=['itemID', 'group_backwards'])

CPU times: user 56.1 s, sys: 362 ms, total: 56.4 s
Wall time: 56.2 s


In [12]:
# LGBM Says on docs that it automatically handles zero values as NaN,
# so we'll keep this standard...
shifting.fillna(0, inplace=True)

<hr>

## Maximum error
The maximum error we could get in this dataset would be just guessing the mean of our sales from weeks 1 to 12, and that's what the cell below is computing.

In [13]:
worst_possible_prediction = shifting.loc[shifting.group_backwards > 1]['orderSum'].mean()
prediction = np.full(shifting.loc[shifting.group_backwards == 1]['orderSum'].shape, worst_possible_prediction) # Array filled with the mean...
target = shifting.loc[shifting.group_backwards == 1]['orderSum']
print("Guessing the mean of 'orderSum' for all items in target", mse(target, prediction) ** 0.5)

Guessing the mean of 'orderSum' for all items in target 118.20105838913783


<hr>

## Dataset Splitting (Train until week 3 / Val. week 2/ Test week 1)
All my experiments will use weeks 13 to 3 as a train set, week 2 as our validation set and week 1 as a test set.

In [14]:
train = shifting.loc[shifting.group_backwards >= 3]
val = shifting.loc[shifting.group_backwards == 2]
test = shifting.loc[shifting.group_backwards == 1]

weights = infos.set_index('itemID')['simulationPrice'].to_dict()

w_train = train['itemID'].map(weights)
w_val = val['itemID'].map(weights)

In [15]:
# I recommend to the other members of the team keeping the
# datatypes of our datasets as Pandas DataFrames instead of Numpy,
# since It will easier to use Boosting Analysis frameworks
y_train = train['orderSum']
y_val = val['orderSum']
X_train = train.drop(columns=["orderSum"])
X_val = val.drop(columns=["orderSum"])

In [16]:
params = {
#           "objective" : "poisson",
          "objective" : "l1",
          "metric" :"rmse",
          "learning_rate" : 0.5,
          'verbosity': 1,
          'max_depth': 6,
          'num_leaves': 32,
          "min_data_in_leaf":3000,
         }

lgbtrain = lgb.Dataset(X_train, label = y_train, weight=w_train, 
                       categorical_feature=[2, 3, 5, 6, 7, 9])
lgbvalid = lgb.Dataset(X_val, label = y_val, weight=w_val, categorical_feature=[2, 3, 5, 6, 7, 9])

num_round = 1000
model = lgb.train(params,
                  lgbtrain,
                  num_round,
                  valid_sets = [lgbtrain, lgbvalid], 
                  verbose_eval=5,
                  early_stopping_rounds=5,
#                   fobj=objective,
                  feval=feval,
                 )

/home/bcoelho/.local/lib/python3.6/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 5 rounds
[5]	training's rmse: 39.4147	training's feval: 5.28139e+06	valid_1's rmse: 44.5538	valid_1's feval: 303195
[10]	training's rmse: 38.5596	training's feval: 8.13172e+06	valid_1's rmse: 43.6461	valid_1's feval: 607847
[15]	training's rmse: 38.2531	training's feval: 8.88644e+06	valid_1's rmse: 43.3013	valid_1's feval: 719913
Early stopping, best iteration is:
[11]	training's rmse: 38.2505	training's feval: 8.92272e+06	valid_1's rmse: 43.2983	valid_1's feval: 721535


<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

Now we create a second model just for the new items.

In [17]:
# Weekpair is negative, so this works:
first_fortnight_item = orders.sort_values("group_backwards",
                                     ascending=False)\
                          .groupby(["itemID"])["group_backwards"].first()
first_fortnight_item = first_fortnight_item.reset_index()
first_fortnight_item.head()

,itemID,group_backwards
0,1,12
1,2,9
2,3,13
3,4,12
4,5,13


In [18]:
train2 = pd.merge(first_fortnight_item, train, on=["itemID", "group_backwards"],
                  how="inner", validate="1:1")
val2 = pd.merge(first_fortnight_item, val, on=["itemID", "group_backwards"],
                  how="inner", validate="1:1")
test2 = pd.merge(first_fortnight_item, test, on=["itemID", "group_backwards"],
                  how="inner", validate="1:1")

# Check we didn't make mistakes...
assert len(train2) + len(val2) + len(test2) == len(first_fortnight_item)
assert len(first_fortnight_item.query("group_backwards >= 3")) == len(train2)
assert len(first_fortnight_item.query("group_backwards == 2")) == len(val2)
assert len(first_fortnight_item.query("group_backwards == 1")) == len(test2)

len(train2), len(val2), len(test2)

(8385, 727, 728)

In [19]:
w_train2 = train2['itemID'].map(weights)
w_val2 = val2['itemID'].map(weights)

In [20]:
y_train2 = train2['orderSum']
y_val2 = val2['orderSum']
y_test2 = test2['orderSum']
# Maybe other features don't make sense
X_train2 = train2.drop(columns=["orderSum", "itemID", "is_new"])
X_val2 = val2.drop(columns=["orderSum", "itemID", "is_new"])
X_test2 = test2.drop(columns=["orderSum", "itemID", "is_new"])
# Make sure to change the categorical features if you drop more cols
cat_feats = [1, 2, 4, 5, 6]

In [21]:
X_train2.iloc[:2, cat_feats]

,brand,manufacturer,category1,category2,category3
0,0,1,1,1,1
1,0,2,1,2,1


In [22]:
params2 = {
#           "objective" : "poisson",
          #"objective" : "l1",
          "objective" : "l2", # L2 works MUCH BETTER than L1
          "metric" :"rmse",
          #"learning_rate" : 0.5,
          'verbosity': 1,
          'max_depth': 5,
          #'num_leaves': 32,
          "min_data_in_leaf":3,
         }
lgbtrain2 = lgb.Dataset(X_train2, label=y_train2, weight=w_train2, 
                        categorical_feature=cat_feats)
lgbvalid2 = lgb.Dataset(X_val2, label=y_val2, weight=w_val2, 
                        categorical_feature=cat_feats)

num_round = 1000
model2_val = lgb.train(params2,
                  lgbtrain2,
                  num_round,
                  valid_sets = [lgbtrain2, lgbvalid2], 
                  verbose_eval=5,
                  early_stopping_rounds=5,
#                   fobj=objective,
                  feval=feval,
                 )

Training until validation scores don't improve for 5 rounds
[5]	training's rmse: 75.2973	training's feval: 6.71152e+06	valid_1's rmse: 91.8662	valid_1's feval: 844086
[10]	training's rmse: 69.0586	training's feval: 1.02511e+07	valid_1's rmse: 84.3397	valid_1's feval: 1.03846e+06
[15]	training's rmse: 64.8813	training's feval: 1.22334e+07	valid_1's rmse: 83.8511	valid_1's feval: 1.10257e+06
[20]	training's rmse: 62.4726	training's feval: 1.32229e+07	valid_1's rmse: 84.796	valid_1's feval: 1.14428e+06
Early stopping, best iteration is:
[16]	training's rmse: 64.3179	training's feval: 1.24865e+07	valid_1's rmse: 83.3976	valid_1's feval: 1.11743e+06


/home/bcoelho/.local/lib/python3.6/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [23]:
# What about on our test?

In [24]:
preds2 = model2_val.predict(X_test2)

In [25]:
new_items = test2["itemID"]
# Check to make sure itemIDs are in the correct order
if not new_items.is_monotonic_increasing:
    raise ValueError("ItemIDs not increasing! this will break the infos order!")

prices_new_items = infos.query("itemID in @new_items")["simulationPrice"]
assert len(y_test2) == len(prices_new_items)
baseline_score(preds2, y_test2.values, prices_new_items.values)

1049579.91

# HOLY FUCK


# FUCK THE ORIGINAL MODEL, LET'S JUST PREDICT THE NEW ITEMS

In [26]:
train = shifting.loc[shifting.group_backwards >= 2]
test = shifting.loc[shifting.group_backwards == 1]

w_train = train['itemID'].map(weights)

In [27]:
train2 = pd.merge(first_fortnight_item, train, on=["itemID", "group_backwards"],
                  how="inner", validate="1:1")
test2 = pd.merge(first_fortnight_item, test, on=["itemID", "group_backwards"],
                  how="inner", validate="1:1")

# Check we didn't make mistakes...
assert len(train2) + len(test2) == len(first_fortnight_item)
assert len(first_fortnight_item.query("group_backwards >= 2")) == len(train2)
assert len(first_fortnight_item.query("group_backwards == 1")) == len(test2)

len(train2), len(test2)

(9112, 728)

In [28]:
w_train2 = train2['itemID'].map(weights)

In [29]:
y_train2 = train2['orderSum']
y_test2 = test2['orderSum']
# Maybe other features don't make sense
X_train2 = train2.drop(columns=["orderSum", "itemID", "is_new"])
X_test2 = test2.drop(columns=["orderSum", "itemID", "is_new"])
# Make sure to change the categorical features if you drop more cols
cat_feats = [1, 2, 4, 5, 6]

In [30]:
X_train2.iloc[:2, cat_feats]

,brand,manufacturer,category1,category2,category3
0,0,1,1,1,1
1,0,2,1,2,1


In [31]:
lgbtrain2 = lgb.Dataset(X_train2, label=y_train2, weight=w_train2, 
                        categorical_feature=cat_feats)
model2 = lgb.train(params2,
                  lgbtrain2,
                  model2_val.best_iteration,
                  valid_sets = [lgbtrain2], 
                  verbose_eval=5,
                  early_stopping_rounds=5,
                  feval=feval,
                 )

Training until validation scores don't improve for 5 rounds
[5]	training's rmse: 76.4994	training's feval: 7.31874e+06
[10]	training's rmse: 69.7218	training's feval: 1.11889e+07
[15]	training's rmse: 65.9111	training's feval: 1.31721e+07
Did not meet early stopping. Best iteration is:
[16]	training's rmse: 65.3554	training's feval: 1.34485e+07


/home/bcoelho/.local/lib/python3.6/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [32]:
final_predictions2 = model2.predict(X_test2)

In [33]:
new_items = test2["itemID"]
# Check to make sure itemIDs are in the correct order
if not new_items.is_monotonic_increasing:
    raise ValueError("ItemIDs not increasing! this will break the infos order!")

prices_new_items = infos.query("itemID in @new_items")["simulationPrice"]
assert len(y_test2) == len(prices_new_items)
baseline_score(final_predictions2, y_test2.values, prices_new_items)

1056532.0159999998

# HAHAHAHAHAHAHAHAHAHAHAHA

Legal! Vamos agora usar o model2 para predizer items novos e o model normal para os items não novos...

E juntar os dois

Mas vamos manter a estrutura de treino do model (mais pq to sem tempo pra testar coisas melhoras)

Só preciso retreinar o model original final então:

In [34]:
# os dados de train agora são o treino TREINO
y_train = train['orderSum']
X_train = train.drop(columns=["orderSum"])

In [35]:
params = {
          "objective" : "l1",
          "metric" :"rmse",
          "learning_rate" : 0.5,
          'verbosity': 1,
          'max_depth': 7,
          'num_leaves': 15,
          "min_data_in_leaf":3500,
         }

lgbtrain = lgb.Dataset(X_train, label = y_train, weight=w_train, categorical_feature=[2, 3, 5, 6, 7, 9])

model = lgb.train(params,
                  lgbtrain,
                  model.best_iteration,
                  valid_sets = [lgbtrain], 
                  valid_names = ['train'],
                  verbose_eval=5,
                  early_stopping_rounds=5,
                  feval=feval,
                 )

Training until validation scores don't improve for 5 rounds
[5]	train's rmse: 39.566	train's feval: 6.51117e+06
[10]	train's rmse: 39.1716	train's feval: 7.95597e+06
Did not meet early stopping. Best iteration is:
[11]	train's rmse: 39.1691	train's feval: 7.97428e+06


In [36]:
y_test = test['orderSum']
X_test = test.drop(columns=["orderSum"])
final_predictions = model.predict(X_test)

In [37]:
assert test["itemID"].is_monotonic_increasing
new_items_idx = test["itemID"].isin(new_items)
final_predictions[new_items_idx] = final_predictions2
final_predictions[final_predictions < 0] = 0

<hr>

### Utilities

In [38]:
final_predictions

array([0.78125, 0.5    , 0.5    , ..., 0.     , 0.     , 0.     ])

In [39]:
baseline_score(final_predictions, y_test.values, infos['simulationPrice'])

1140288.454

Old value was ~600 thousand